# Create a Calculator server using socket programming 
 The Server should take string input from the client and use this input and perform the required operation and return the value as an integer 
 <br>
Ex : 
- Client Says "Please calculate 2*4" the server should return "8" 
- The Client could as say "Cal 2*4" and the server should return "8" 
- The client could say "Add 2+4" and the server should return "6"
> Look for the operator in the string and check for integers preceding and following the operator

# Server Code
- Create a Socket at 127.0.0.1 or any other IP and some port no Ex :  8080

In [ ]:
import socket
import re

# Define the function to do the calculation part
def evaluate_expression(expression):
    try:
        result = eval(expression)
        return result
    except Exception as e:
        return f"Error: {e}"

HOST = '127.0.0.1'
PORT = 8080

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind((HOST, PORT))
    s.listen()
    print(f"Server listening on {HOST}:{PORT}")

    conn, addr = s.accept()
    with conn:
        print(f"Connected by {addr}")
        while True:
            try:
                # Receive data from the client
                data = conn.recv(1024)
                if not data:
                    print("No data received, closing connection.")
                    break  # Client disconnected
                print(f"Received data: {data.decode()}")

                # Process the message
                message = data.decode()

                # Check if the message contains a valid command
                if any(command in message.lower() for command in ["calculate", "cal", "add", "subtract", "sub", "multi", "multiply", "div", "divide"]):
                    expression = message.split(maxsplit=1)[1]
                else:
                    expression = message.strip()

                # Sanitize the expression
                expression = re.sub(r"[^0-9+\-*/().]", "", expression)

                # Evaluate the expression and get the result
                result = evaluate_expression(expression)

                # Send the result back to the client
                conn.sendall(str(result).encode())

            except Exception as e:
                # If there's an error handling the message, send an error message to the client
                print(f"Error handling message: {e}")
                conn.sendall(f"Error: {e}".encode())



Server listening on 127.0.0.1:8080
Connected by ('127.0.0.1', 52067)
Received data: calc 3*7


# Client Code 
# ===========
- Use a Separate file for this as 2 cells can't simultaneously run in a Python Notebook 
- The Client must connect to the sever at this socket 
- Take input from Serial Monitor and send it to server 

In [1]:
import socket

HOST = '127.0.0.1'  
PORT = 8080         


with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    try:
    
        s.connect((HOST, PORT))
        print(f"Connected to server at {HOST}:{PORT}")
        
        while True:
            message = input("Enter a mathematical expression or command (e.g., 'add 3+4', 'calculate 5*5', 'exit'): ")

            if message.lower() == 'exit':
                print("Exiting client.")
                break

            s.sendall(message.encode())

            data = s.recv(1024)

            print(f"Server response: {data.decode()}")

    except Exception as e:
        print(f"Error connecting to the server: {e}")


Error connecting to the server: [WinError 10061] No connection could be made because the target machine actively refused it


# Try running 2 client python file separately in 2 terminals 
- What happens to the output ? 
With the current code,the output disappears. This is because the server code uses s.accept(), which blocks and waits for one client to connect at a time. When one client connects, the server is blocked in the while loop until that client disconnects or the connection is closed.